In [ ]:
import requests 
from bs4 import BeautifulSoup
import json
from urllib.parse import urlparse
import pandas as pd 
import numpy as np 



def catchvalue(get):
    newword=str(get).replace("product_d.php?","").split('&')
    twnew=newword[0].replace("lang=","")  
    tbnew=newword[1].replace("tb=","")
    idnew=newword[2].replace("id=","")
    
    result={'lang':f'{twnew}',
            'tb':f'{tbnew}',
            'id':f'{idnew}',
            'type':'productdata'}
    return result

    
#找每種類別的最終頁數
def allpage(f):      
    lst=[]
    for i in f:
        if "page_last" in str(i):            
            lst.append(i.find('a').get('href'))   #取到所有類別的最後頁數
            my_str=lst[0]
            my_str=my_str.replace("product.php?","")
            newstr=my_str.replace("product_d.php?","").split('&')
            new2=newstr[2].replace("cid=" , "")
            return new2

#每個分類的cid 對應頁數/分類            
#高跟 cid =8 /16  
#低跟 cid=137 / 4
#涼鞋 cid=14 /3
#靴子 cid=129 /3
#平底 cid=11 /6
#男鞋 cid=121/3
#童鞋 cid=17/3


def Returnpage(cid):
    if cid =="8":
        return ["16","高跟"]    
    if cid =="137":
        return ["4","低跟"]
    if cid =="14":
        return ["3","涼鞋"]
    if cid =="129":
        return["3","靴子"]
    if cid =="11":
        return ["6","平底"]
    if cid =="121":
        return ["3","男鞋"]
    if cid =="17":
        return ["3","童鞋"]  

frame = []    

url ='https://www.momentanee.com/index.php?lang=tw'

re=requests.get(url)
soup=BeautifulSoup(re.text ,'html.parser')
headers = {
    'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
}

menu=soup.find('div',class_='dropdown-menu SP').find_all('a')


diftype =dict()
#各種類
for each in menu[1:8]:   
    title =each.text
    href = 'https://www.momentanee.com/'+ each.get('href')
    diftype[title]=href

        
#各分類裡的資料
for title,href in diftype.items():
 
    eachsite=requests.get(href)
    sp=BeautifulSoup(eachsite.text,'html.parser')



    #同一品項總共頁數
    pages=sp.find('div',class_='page').findNext('ul').find_all('li')
    eachcid=allpage(pages)
    matchpage=Returnpage(eachcid)
      
    #得到全部款式的所有頁的url   
    for j in range(1,int(matchpage[0])+1):  
        allurl=f"https://www.momentanee.com/product.php?lang=tw&tb=1&cid={eachcid}&ot=all&currentpage={j}"

       
        request=requests.get(allurl)
        BS=BeautifulSoup(request.text , 'html.parser')

    
        #單一類別的所有鞋款
        product=BS.find('div', class_="panel_3 container" )
        items=product.find_all('div',class_='col-md-3 col-sm-4 col-xs-6')

        info= []
            
        for i in items:
            result = 'https://www.momentanee.com/'+ i.find('a').get('href')           
 
            re = requests.get(result)
            sb = BeautifulSoup(re.text , 'html.parser')
        

            newhref=i.find('a').get('href')
            catch=catchvalue(newhref)

            newweb=requests.post('https://www.momentanee.com/include/shopping_process.php',data = catch)
            APItext=json.loads(newweb.text)      
           
            if APItext['standskeys'] == []:
                
                heel=sb.find('div',class_='products-info')

                #鞋款名字
                name=heel.find('div',class_='name').text
                #產品描述   
                if "editor clearfix" not in str(heel):
                    pass
                else:
                    describe=sb.find('article',class_="editor clearfix").text.replace('\n','').replace('門市資訊','').lstrip()
                    describe="".join(describe.split())

                    if len(describe) == 0:   
                        describe = "沒有資料"
                    else:
                        describe

                
                data = {'category':matchpage[1],'name':name,'describe':describe,'price':price,'size':str(np.random.randint(31,43)),'stock':str(np.random.randint(1,40)),'count':str(np.random.randint(1,25))}
                frame.append(data)
                
            else:
                for i in APItext['standskeys']:

                    heel=sb.find('div',class_='products-info')
                    
                    #鞋款名字
                    name=heel.find('div',class_='name').text
                    #售價
                    price= APItext['stands'][f'{i}']['price']

                    #產品描述   
                    if "editor clearfix" not in str(heel):
                        pass
                    else:
                        describe=sb.find('article',class_="editor clearfix").text.replace('\n','').replace('門市資訊','').lstrip()
                        describe="".join(describe.split())
                            
                        if len(describe) == 0:   
                            describe = "沒有資料"
                        else:
                            describe
                    #尺寸
                    size = APItext['stands'][f'{i}']['standName']
                    #產品庫存
                    stock  = APItext['stands'][f'{i}']['inventory']
                    #數量
                    count = APItext['stands'][f'{i}']['inventorySet']
                    
                    data = {'category':matchpage[1],'name':name,'describe':describe,'price':price,'size':size,'stock':stock,'count':count}
                    frame.append(data)
print(frame)